In [1]:
# %pip install --upgrade --quiet  pinecone-client langchain-openai tiktoken langchain

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get the API key
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
# Get the PINECONE_ENV
os.environ["PINECONE_ENV"] = os.getenv("PINECONE_ENV")

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings

In [12]:
import PyPDF2

def convert_pdf_to_txt(pdf_path, txt_path):
    # Open the PDF file in read-binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Initialize an empty string to store the text
        text = ''

        # Loop through each page in the PDF
        for page_num in range(len(pdf_reader.pages) ):
            # Extract the text from the page
            text += pdf_reader.pages[page_num].extract_text()


    # Write the text to a text file
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)
convert_pdf_to_txt('10_Academy_challenge_doc.pdf', '10_Academy_challenge_doc.txt')

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("10_Academy_challenge_doc.txt", encoding = 'UTF-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [12]:
from pinecone import Pinecone, PodSpec

# initialize pinecone
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

index_name = "10-academy-challenge"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pc.list_indexes():
    # we create a new index
    pc.create_index(name=index_name, metric="cosine", dimension=1536, spec=PodSpec(
                environment="gcp-starter"
            ))
            
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone(docs, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What is this week's challenge?"
docs = docsearch.similarity_search(query)

AttributeError: 'OpenAIEmbeddings' object has no attribute 'startswith'

In [9]:
index = pc.Index("10-academy-challenge")
vectorstore = Pinecone(index, embeddings.embed_query, docs)

vectorstore.add_texts("More text!")

TypeError: config must be of type pinecone.config.Config

In [10]:
retriever = docsearch.as_retriever(search_type="mmr")
matched_docs = retriever.get_relevant_documents(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)

NameError: name 'docsearch' is not defined